In [47]:
import sqlite3
import pandas as pd
import json

In [21]:
DATABASE = "./recruit.db"

In [22]:
conn = sqlite3.connect(DATABASE)

In [99]:
education = pd.read_sql('SELECT * FROM education', conn)
race = pd.read_sql('SELECT * FROM race', conn)
insurance_segment = pd.read_sql('select * FROM insurance_segment', conn)
customer = pd.read_sql('select * FROM customer', conn)

query = 'SELECT customer.economic_stability, insurance_segment.value as insurance_segment, count(customer.economic_stability) as cnt FROM customer GROUP BY customer.economic_stability, insurance_segment.value INNER JOIN insurance_segment ON insurance_segment.id = customer.insurance_segment_id'
query = 'SELECT customer.economic_stability, insurance_segment.value, count(customer.economic_stability) as cnt FROM customer INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id GROUP BY customer.economic_stability, insurance_segment.value'


test = pd.read_sql(query, conn)

In [95]:
test.groupby('economic_stability').sum()['cnt'].max()

4959

In [100]:
test = test.pivot(index = 'economic_stability', columns = 'value', values = 'cnt').fillna(0)

In [101]:
test.reset_index()

value,economic_stability,Country Middles,Full House,Getting By,Hands Full,Price Isn't Primary,Solid Suburbans,Stretched Singles,The Nest is Empty
0,01,0.0,887.0,7.0,7.0,97.0,176.0,9.0,10.0
1,02,7.0,1399.0,39.0,26.0,245.0,483.0,7.0,10.0
2,03,14.0,1493.0,68.0,55.0,378.0,783.0,24.0,57.0
3,04,36.0,1589.0,90.0,103.0,527.0,1157.0,23.0,93.0
4,05,44.0,1488.0,107.0,141.0,599.0,1253.0,18.0,116.0
5,06,110.0,1510.0,105.0,174.0,685.0,1383.0,27.0,143.0
6,07,145.0,1358.0,137.0,204.0,774.0,1260.0,37.0,224.0
7,08,215.0,1537.0,137.0,253.0,817.0,1311.0,41.0,270.0
8,09,287.0,1473.0,176.0,253.0,856.0,1252.0,44.0,296.0
9,10,363.0,1498.0,189.0,338.0,853.0,1225.0,48.0,338.0


In [48]:
customerjson = json.loads(customer.to_json(orient='split'))

In [30]:
def dictfetchall(cursor):
    "Return all rows from a cursor as a dict"
    columns = [col[0] for col in cursor.description]
    return [
        dict(zip(columns, row))
        for row in cursor.fetchall()
    ]


def query_db(query):

    result_dict = {}

    try:
        connection = sqlite3.connect(DATABASE)

        cursor = connection.cursor()
        cursor.execute(query)
        result_dict = dictfetchall(cursor)

    except sqlite3.OperationalError as e:
        print("Db operation error", e)
        result_dict["error"] = str(e)
    except:
        e = sys.exc_info()[0]
        print("An error occurred with the database", e)
        result_dict["error"] = str(e)
    else:
        cursor.close()
        connection.close()

    return result_dict

In [52]:
customerjson['data'][1]

[2,
 'W',
 2.0,
 'O',
 'FL',
 ' ',
 ' ',
 ' ',
 357,
 136,
 None,
 '08',
 6,
 '19',
 '18',
 'M']

In [58]:
customer.head()

,id,race_code,education_id,home_owner,state,is_smoker,is_exerciser,has_insurance,income,travel_spending,sports_leisure_spending,economic_stability,insurance_segment_id,youtube_user_rank,facebook_user_rank,gender
0,1,B,1.0,,NE,,,,25,33,NaN,27,1,10,10,M
1,2,W,2.0,O,FL,,,,357,136,NaN,08,6,19,18,M
2,3,O,NaN,O,NE,,1,,25,18,5.0,13,1,19,19,M
3,4,O,2.0,,MO,,1,,51,527,NaN,23,6,14,16,M
4,5,O,NaN,,MO,,1,,51,527,NaN,23,6,14,16,F


In [86]:
insurance_segment.to_dict(orient='records')

[{'id': 1, 'value': "Price Isn't Primary"},
 {'id': 2, 'value': 'Stretched Singles'},
 {'id': 3, 'value': 'Hands Full'},
 {'id': 4, 'value': 'The Nest is Empty'},
 {'id': 5, 'value': 'Country Middles'},
 {'id': 6, 'value': 'Solid Suburbans'},
 {'id': 7, 'value': 'Full House'},
 {'id': 8, 'value': 'Getting By'}]

In [70]:
import colorlover as cl

In [77]:
cl.scales['8']['qual']['Set1']

['rgb(228,26,28)',
 'rgb(55,126,184)',
 'rgb(77,175,74)',
 'rgb(152,78,163)',
 'rgb(255,127,0)',
 'rgb(255,255,51)',
 'rgb(166,86,40)',
 'rgb(247,129,191)']

In [81]:
connection = sqlite3.connect(DATABASE)
cursor = connection.cursor()
query = 'SELECT customer.economic_stability, insurance_segment.value, count(customer.economic_stability) as cnt \
            FROM customer \
            INNER JOIN insurance_segment ON customer.insurance_segment_id = insurance_segment.id \
            GROUP BY customer.economic_stability, insurance_segment.value'
cursor.execute(query)


In [ ]:
data = dict()
for row in cursor.fetchall():
    if 

In [83]:
data = dict()
for col in [col[0] for col in cursor.description]:
    

(('economic_stability', None, None, None, None, None, None),
 ('value', None, None, None, None, None, None),
 ('cnt', None, None, None, None, None, None))

In [84]:
a = dict()
a['hello']['goodbye'] = 5

KeyError: 'hello'

In [96]:
test.reset_index()

,index,economic_stability,value,cnt
0,0,01,Full House,887
1,1,01,Getting By,7
2,2,01,Hands Full,7
3,3,01,Price Isn't Primary,97
4,4,01,Solid Suburbans,176
5,5,01,Stretched Singles,9
6,6,01,The Nest is Empty,10
7,7,02,Country Middles,7
8,8,02,Full House,1399
9,9,02,Getting By,39
